In [ ]:
!python --version

In [ ]:
import os, sys
from scipy import stats
import numpy as np
import pandas as pd

from scipy import stats
is_colab = False

if is_colab:
    # create src and upload stat_lib into it
    from src import stat_lib
else:
    sys.path.insert(1, '../src/')
    import stat_lib

import seaborn as sns
import matplotlib.pyplot as plt
# %matplotlib inline

### Versão do SciPy

In [ ]:
import scipy

scipy.__version__

### 📌 Kruskal–Wallis - stats.wilcoxon()

O **teste de Kruskal–Wallis:** é um `teste não paramétrico` dual da ANOVA.

Premissas básicas:

  - É ANOVA baseada em ranks
  - Funciona com dados não normais
  - Requer independência
  - 3 ou mais grupos independentes
  - Estatística H ~ qui-quadrado

#### Funcionamento

  1. Junta todos os valores
  2. Ordena todos os dados concatenados
  3. Substitui valores por ranks
  4. Soma ranks por grupo
  5. Compara se as somas diferem mais do que o esperado pelo acaso

#### 🧪 Hipótese nula:

𝐻0: As distribuições populacionais dos k grupos são idêntica

H0: F1(x) == F2(x) == ... == Fk(x)

  - O teste não testa diretamente médias
  - O teste não testa diretamente medianas
  - Os ranks médios são iguais entre os grupos.

https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.mannwhitneyu.html

### Refazendo o exemplo da aula de ANOVA

  - dadas 5 amostras in silico
  - com media 140 e variando com delMU
  - com SSD 10, variando com delSSD

In [ ]:
samp_list=[]; mu_list = []; ssd_list = []
N   = 30
n_samp = 5

MU = 140; delMU = -2
SSD = 10; delSSD = -.1

for i in range(n_samp):
    np.random.seed(12)
    samples = np.random.normal(loc=MU, scale=SSD, size=N)

    samp_list.append(samples)
    MU += delMU
    SSD += delSSD

    mu_list.append(np.mean(samp_list[i]))
    ssd_list.append(np.std(samp_list[i]))

    ret, text, text_stat, stat, pvalue = stat_lib.calc_normalidade_SWT(samples, 0.05, NS='---')
    print(i, text)
    print(f"N={N} {text_stat}")
    print()

In [ ]:
fig, ax = plt.subplots(figsize=(12, 6))

seqx = np.linspace(70, 180, 100)
colors = ['red', 'blue', 'green', 'brown', 'black']

for i in range(n_samp):

    samples = samp_list[i]
    color = colors[i]

    label = f"{color} {mu_list[i]:.1f} ({ssd_list[i]:.1f})"

    plt.vlines(mu_list[i], 0, 0.045, color=color)

    normal_pdf = stats.norm.pdf(seqx, mu_list[i], ssd_list[i])
    sns.lineplot(x=seqx, y=normal_pdf, label=label, color=color)

title = 'Distribuições'
plt.legend()
plt.grid()
plt.legend()
plt.title(title);


In [ ]:
ret, text, text_stat, stat, pvalue = \
stat_lib.test_one_way_ANOVA_list(samp_list)

text, text_stat

### Se diminuir o N não posso (não devo) usar ANOVA

In [ ]:
samp_list=[]; mu_list = []; ssd_list = []
N   = 6
n_samp = 5

MU = 140; delMU = -7
SSD = 10; delSSD = -.3

for i in range(n_samp):
    np.random.seed(12)
    samples = np.random.normal(loc=MU, scale=SSD, size=N)

    samp_list.append(samples)
    MU += delMU
    SSD += delSSD

    mu_list.append(np.mean(samp_list[i]))
    ssd_list.append(np.std(samp_list[i]))

    ret, text, text_stat, stat, pvalue = stat_lib.calc_normalidade_SWT(samples, 0.05, NS='---')
    print(i, text)
    print(f"N={N} {text_stat}")
    print()

### Não adianta plotar a distribuição teoria pois ela não espelha N=6, KDE sim

In [ ]:
df = stat_lib.join_series_by_list(samp_list)
# 5 grupos * 6 amostras
len(df)

In [ ]:
df.head(3)

In [ ]:
df.tail(3)

### KED plot - mostra melhor a distorção de amostrar N=6

In [ ]:
sns.displot(df, x="val", hue="grupo", kind="kde", fill=True, height=4, aspect=1.4)

title = 'Distribuições por grupos'
plt.grid()
plt.xlabel('grupos e valores')
plt.ylabel('densidade')
plt.title(title);

In [ ]:
### 

In [ ]:
# ou lista as diversas amostras ou 
# coloca um * indicando que é um ponteiro para lista (endereço)
stat, pval = stats.kruskal(*samp_list)

print("H =", stat)

if pval >= 0.05:
    print(f"Aceita-se H0, p-value = {pval:.1e}")
else:
    print(f"Rejeita-se H0, p-value = {pval:.1e}")
    

### Teste de Dunn

Após Kruskal–Wallis -> Teste de Dunn

  - Faz comparações par a par
  - Baseado em ranks
  - Usa correção múltipla (Bonferroni, Holm, etc.)
  - É o análogo não paramétrico do Tukey.

p_adjust - técnica de ajustar o p-valor devido a testes com repetição:

 1. bonferroni
 2. holm
 3. fdr_bh
 4. sidak


In [ ]:
# se não tiver esta library/pacote - instale
# !pip install scikit_posthocs

In [ ]:
import scikit_posthocs as sp

dfr = sp.posthoc_dunn(df, val_col='val', group_col='grupo', p_adjust='bonferroni')

dfr